In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import os
import json
import dotenv

In [3]:
dotenv.load_dotenv()

True

In [4]:

from langchain_deepseek import ChatDeepSeek
from langchain_core.tools import tool
from langchain_core.runnables import RunnableConfig
from langgraph.prebuilt import create_react_agent, InjectedState
from langgraph.graph.message import add_messages

In [5]:
from src.utils_stock import get_top_nasdaq_stock_data
from src.utils_news import search_and_scrape_url
from src.utils_email import send_email_gmail, markdown_to_html

In [6]:
# parameter
NUM_DAYS = 10
EMAIL_ADDRESS = "nelsonlin0321@gmail.com"

In [7]:
from typing import Annotated, TypedDict


class CustomState(TypedDict):
    messages: Annotated[list, add_messages]
    number_days_of_analysis: int
    email: str

In [8]:
def get_system_prompt(suffix: str) -> str:
    return (
        "You are a helpful AI assistant, collaborating with other assistants."
        " Use the provided tools to progress towards answering the question."
        " If you are unable to fully answer, that's OK, another assistant with different tools "
        " will help where you left off. Execute what you can to make progress."
        " If you or any of the other assistants have the final answer or deliverable,"
        " prefix your response with FINAL ANSWER so the team knows to stop."
        f"\n{suffix}"
    )

In [9]:
llm = ChatDeepSeek(model="deepseek-chat",
                   api_key=os.getenv("DEEPSEEK_API_KEY"))

### Data Acquisition Agent


In [10]:
@tool
def get_top_nasdaq_performance_stock_data(
        config: Annotated[dict, RunnableConfig]) -> str:
    """Get the day's top NASDAQ-100 gainer with symbol, percentage increase, sample data in markdown format, pandas dataframe info and csv data path for further python code analysis.
    """
    print(config.keys())
    result = get_top_nasdaq_stock_data(
        config["configurable"]["number_days_of_analysis"])
    return json.dumps(result)

In [11]:
# Research agent and node
data_acquisition_agent = create_react_agent(
    llm,
    tools=[get_top_nasdaq_performance_stock_data],
    prompt=get_system_prompt(
        "You are a data acquisition specialist focused on retrieving NASDAQ market data. Your task is to fetch information about today's top performing NASDAQ stock using the get_top_nasdaq_performance_stock_data tool. The data should include the stock symbol, percentage gain, and relevant trading data for analysis."
    ),
)

### Python Data Analysis Agent


In [12]:
from langchain_experimental.utilities import PythonREPL

In [13]:
from typing import Annotated


repl = PythonREPL()


@tool
def python_repl_tool(
    code: Annotated[str, "The python code to execute to do comprehensive stock performance analysis"],
):
    """Use this to execute python code. If you want to see the output of a value,
    you should print it out with `print(...)`. This is visible to the user."""
    try:
        print(f"Executing code:\n{code}")
        result = repl.run(code)
    except BaseException as e:
        return f"Failed to execute. Error: {repr(e)}"
    result_str = f"Successfully executed:\n```python\n{code}\n```\nStdout: {result}"
    return (
        result_str + "\n\nIf you have completed all tasks, respond with FINAL ANSWER."
    )

In [14]:
python_data_analysis_agent = create_react_agent(
    llm,
    [python_repl_tool],
    prompt=get_system_prompt(
        """
        You are an agent designed to write and execute python code for comprehensive stock analysis given data. 
        The data include the stock symbol, percentage increase, sample data in markdown format, pandas dataframe info, and csv data path for python code analysis. 
        
        You have access to a python REPL, which you can use to execute python code.
        If you get an error, debug your code and try again.

        
        Your tasks are to generate code and execute codes with python REPL to fullfil below requirements:
        1. Data Loading and Preparation:
           - Read and clean the provided CSV data using pandas

        2. Technical Analysis:
           - Calculate and analyze moving averages (5-day, 10-day)
           - Compute daily returns and cumulative returns
           - Calculate trading volume trends
           - Determine price momentum indicators
           - Analyze last n trading days performance (average change, trend direction)
           - Calculate volatility metrics (standard deviation, beta)

        3. Statistical Insights:
           - Perform descriptive statistics on price and volume data
           - Calculate risk metrics (Sharpe ratio if applicable)
           - Identify any statistically significant patterns
           - Analyze distribution of returns

        All the result should be using python print() to present in a clear, structured format using print statements so that is visible to the user in the console.
        Ensure all the tasks are completed and the final result is clear and concise.
        Do not ask the confirmation to completed future tasks that may required. 
"""
    ),
)

### News Research Agent


In [15]:
@tool
def get_stock_news(query: Annotated[str, "The query to get the stock financial news"]) -> str:
    """Use this to do google search news to perform stock news analysis. The results tool will return the news_url with content for your analysis"""
    search_result = search_and_scrape_url(query)
    return json.dumps(search_result)

In [16]:
news_research_agent = create_react_agent(
    llm,
    [get_stock_news],
    prompt=get_system_prompt(
        """
        You are an agent designed to do comprehensive stock news research using google news search.
        Your tasks include:
        1. Sentiment Analysis:
           - Analyze the overall market sentiment (bullish/bearish)
           - Identify key positive and negative factors affecting the stock
           - Evaluate the tone of recent news coverage
        
        2. Risk Assessment:
           - Identify potential risks and challenges mentioned in news
           - Analyze regulatory or legal concerns
           - Evaluate market competition and industry dynamics
        
        3. News Impact Analysis:
           - Assess how recent news might affect stock price
           - Look for patterns in news coverage
           - Identify significant company events or announcements
        
        4. Market Context:
           - Consider broader market conditions
           - Analyze sector-specific news and trends
           - Compare with competitor news and performance
        
        Provide clear, structured insights based on the news data to support investment decisions.
        """
    ),
)

### Reporting Agent


In [17]:
@tool
def send_report_via_email(
        subject: Annotated[str, "Email subject of report for email sending"],
        body: Annotated[str, "Report content in markdown from reporting agent for email sending"],
        state: Annotated[dict, InjectedState],
) -> str:
    """
    Use this tool to send the generated report.

    Args:
        subject (Annotated[str, &quot;Email subject of report for email sending&quot;]): _description_
        body (Annotated[str, &quot;Report content in markdown from reporting agent for email sending&quot;]):

    Returns:
        str: description
    """
    bod_in_html = markdown_to_html(body)
    result = send_email_gmail(subject, bod_in_html, state['email'])
    return result

In [18]:
reporting_agent = create_react_agent(
    llm,
    tools=[send_report_via_email],
    prompt=get_system_prompt(
        """
        You are a professional financial analyst tasked with synthesizing data and creating comprehensive stock analysis reports.
        Firstly, please write the report based on below requirements and you're require to use the send_report_via_email tool with subject and report as the email body.
        
        Your role is to:
        1. You must analyze and integrate findings from two sources:
           - Technical analysis from the python_data_analysis_agent
           - News and sentiment analysis from the news_research_agent
        
        2. Structure your report in two clear sections:
           - Technical Analysis: Interpret and explain the quantitative data findings
           - Market Research & Sentiment: Present key insights from news analysis
        
        3. For each section:
           - Highlight the most significant findingsd
           - Explain their implications for the stock
           - Support conclusions with specific data pointsd
        
        4. Conclude with:
           - Key takeaways that combine both technical and news analysis
           - Notable risks or opportunities identified
        
        Write in a clear, professional style. Focus on actionable insights.
        Present the report directly without any introductory text or meta-commentary.

        Please don't ask the confirmation sending this report via email. Sending the report once after you finish the reporting.
        """
    ),
)

### Define Graph


In [19]:
from typing import Literal
from langgraph.graph import END
from langgraph.types import Command
from langchain_core.messages import HumanMessage

In [20]:
# def get_next_node(last_message: BaseMessage, goto: str):
#     if "FINAL ANSWER" in last_message.content:
#         # Any agent decided the work is done
#         return END
#     return goto

In [21]:
def data_acquisition_agent_node(
    state: CustomState,
) -> Command[Literal["python_data_analysis_agent"]]:   # Go to python_data_analyst node
    result = data_acquisition_agent.invoke(state)
    result["messages"][-1] = HumanMessage(
        content=result["messages"][-1].content, name="data_acquisition_agent"
    )
    return Command(
        update={
            "messages": result["messages"],
        },
        goto="python_data_analysis_agent",
    )

In [22]:
def python_data_analysis_agent_node(
    state: CustomState,
) -> Command[Literal["news_research_agent"]]:
    result = python_data_analysis_agent.invoke(state)
    result["messages"][-1] = HumanMessage(
        content=result["messages"][-1].content, name="python_data_analysis_agent"
    )
    return Command(
        update={
            "messages": result["messages"],
        },
        goto="news_research_agent",
    )

In [23]:
def news_research_agent_node(
    state: CustomState,
) -> Command[Literal["reporting_agent"]]:  # type: ignore
    result = news_research_agent.invoke(state)
    result["messages"][-1] = HumanMessage(
        content=result["messages"][-1].content, name="news_research_agent"
    )
    return Command(
        update={
            "messages": result["messages"],
        },
        goto="reporting_agent",
    )

In [24]:
def reporting_agent_node(
    state: CustomState,
) -> Command[Literal[END]]:  # type: ignore
    result = reporting_agent.invoke(state)
    result["messages"][-1] = HumanMessage(
        content=result["messages"][-1].content, name="reporting_agent"
    )
    return Command(
        update={
            "messages": result["messages"],
        },
        goto=END,
    )

In [25]:
from langgraph.graph import StateGraph, START

workflow = StateGraph(CustomState)
workflow.add_node("data_acquisition_agent", data_acquisition_agent_node)
workflow.add_node("python_data_analysis_agent",
                  python_data_analysis_agent_node)
workflow.add_node("news_research_agent", news_research_agent_node)
workflow.add_node("reporting_agent", reporting_agent_node)

workflow.add_edge(START, "data_acquisition_agent")
# workflow.add_edge("data_acquisition_specialist", "python_data_analyst")
graph = workflow.compile()

In [26]:
graph.get_graph().print_ascii()

        +-----------+          
        | __start__ |          
        +-----------+          
               *               
               *               
               *               
  +------------------------+   
  | data_acquisition_agent |   
  +------------------------+   
               .               
               .               
               .               
+----------------------------+ 
| python_data_analysis_agent | 
+----------------------------+ 
               .               
               .               
               .               
    +---------------------+    
    | news_research_agent |    
    +---------------------+    
               .               
               .               
               .               
      +-----------------+      
      | reporting_agent |      
      +-----------------+      
               .               
               .               
               .               
          +---------+          
        

In [27]:
events = graph.stream(
    {
        "messages": [{"role": "user", "content": "First, Let get the top performance stock data."}],
    },
    config={"configurable": {"thread_id": "1",
                             "number_days_of_analysis": NUM_DAYS, "email": EMAIL_ADDRESS, }},
    stream_mode="values",

)
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()
    # print(s)
    # print("----")

================================ Human Message =================================

First, Let get the top performance stock data.
================================ Human Message =================================
Name: data_acquisition_agent

It seems the function is not working as expected, possibly due to a technical issue or misconfiguration. Let me know if you'd like me to try an alternative approach or if there's another way you'd like to proceed. 

Alternatively, if you have access to another tool or method for retrieving the top-performing NASDAQ stock data, I can assist with that.


KeyboardInterrupt: 